In [ ]:
import pandas as pd
import os
import wandb
DATA_PATH ='/Users/favea/Downloads/swiss-data'

### READ DATASETS

In [ ]:

df_delay = pd.read_csv(os.path.join(DATA_PATH,'delay.csv'), delimiter=';', dtype={'wh_fdel_delay_subcode':str})
df_fis = pd.read_csv(os.path.join(DATA_PATH,'fis.csv'), delimiter=';')
df_delay = df_delay.drop_duplicates(subset='wh_fdel_leg_i')

## MERGE DATASET

In [ ]:
merged_df = df_fis.merge(df_delay, left_on='wh_fleg_leg_i', right_on='wh_fdel_leg_i')


In [ ]:
pd.set_option('display.max_rows', None)
merged_df[merged_df.wh_fleg_flt_legs > 1].sample(10).T

In [ ]:
merged_df['is_delayed'] = merged_df.wh_fdel_delay_time>=15



In [ ]:
merged_df.groupby('is_delayed').count()

In [ ]:
pd.set_option('display.max_rows', None)
df_fis[(df_fis.wh_fleg_arr_ap_scd == 'BRI')].T

In [ ]:
df_fis[df_fis.wh_fleg_cycles >= 10].sample(10).T # turn-around flights and "positioning/ferry flights"

In [ ]:

df_fis.sample(100).head(10).T
# PAX information can tell whether there's passengeres that cancelled 
# WH: stands for wheel (position from where the flight is taking off or landing)
# 

In [ ]:
df_fis.groupby(by='wh_fleg_flt_carrier').count() # REMOVE DC = wh_fleg_flt_carrier (only 4 observations)
df_fis.wh_fleg_season = df_fis.wh_fleg_season.str.strip() # Error type with season (I guess is spring-summer and Winter|Autumn)
df_fis.groupby(by='wh_fleg_season').count() # df_fis.wh_fleg_season = df_fis.wh_fleg_season.str.strip() # Error type with season # Only info about Summer and Winter 2021/2022 (also W2020)
df_fis.groupby(by='wh_fleg_leg_state').count() # there's 4242 cancelled flgiths (CLN) , not need to use them. I can remove  them I guess


In [ ]:
df_fis.groupby(by='wh_fleg_dep_ap_scd')[['wh_fleg_leg_i']].count().sort_values(by = 'wh_fleg_leg_i',ascending=False) # most of  the flights flgiht from Zurich or Geneva

In [ ]:
df_fis.groupby(by='wh_fleg_season').count() 



In [ ]:
df_fis.groupby(by=['wh_fleg_dep_ap_scd','wh_fleg_arr_ap_scd'])[['wh_fleg_leg_i']].count().sort_values(by = 'wh_fleg_leg_i',ascending=False)
# The most common itinerary is BER-ZRR; LHR-ZRH  (Hetrow), GVA-ZUrich (baaad for the environment)

In [ ]:
df_fis[df_fis.wh_fleg_flt_carrier == 'LX'].head(10).T

In [ ]:
df_delay.sample()

In [ ]:
df_delay.groupby('wh_fdel_delay_code').count().sort_values(by='wh_fdel_leg_i', ascending=False)

# Most common: 
# 0 (internal err) -> Not easy to understand what is is
# 93 (RA): Aircraft rotation, late arrival of aircraft from another flight or previous sector
# 89 (AM): Restrictions at airport of departure,airport/runway closed due obstruction, industrial action, staff shortage, political unrest, noise abatemen
# 81 (AT): ATC restriction en-route or capacity
# 91 (RL): Passenger or Load Connection, awaiting load or passengers from another flight. Protection of stranded passengers onto a new flight.
# 83 (AE): ATC restriction at destination
# 85 (AS): Mandatory security
# 84 (AW): ATC restriction due to weather at destination
# 16 (PS): Commercial Publicity, Passenger Convenience, VIP, Press, Ground meals and missing personal items
# 2 Delay codes starting with 2 (cargo/mail)
# 41 (TD): Aircraft defects


In [ ]:
df_delay[df_delay.wh_fdel_delay_code == 0].head()

In [ ]:
run = wandb.init(project='swiss-delay-prediction', entity=None, job_type="upload", )
raw_data_at = wandb.Artifact('data', type="raw_data")


In [ ]:
raw_data_at.add_file("/Users/favea/Downloads/swiss-data/delay.csv")
raw_data_at.add_file("/Users/favea/Downloads/swiss-data/fis.csv")

run.log_artifact(raw_data_at)

In [ ]:
run.finish()


In [ ]:
merged_df.to_csv('/Users/favea/Downloads/swiss-data/merged_df.csv')

In [ ]:
with wandb.init(project='swiss-delay-prediction', entity=None, job_type="load_merged_dataset") as run:
    table_merged = wandb.Table(dataframe=merged_df)


    # Create an artifact for our dataset
    dataset_artifact = wandb.Artifact(
        'dataset-merged', type='dataset',
        description='Table containing the Delay CSV dataset merged with Fis dataset on columns: wh_fdel_leg_i and wh_fleg_leg_i',
    )
    # Add the table to the artifact & log the artifact
    dataset_artifact.add(table_merged, 'data-table-delay')
    dataset_artifact.add_file('/Users/favea/Downloads/swiss-data/merged_df.csv')


    # Add the 
    run.log_artifact(dataset_artifact)
    


